<a href="https://colab.research.google.com/github/abhishek203/E-Abhishek/blob/master/Chatbot_3(error).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import re
import time
import io
import os
from keras.preprocessing.sequence import pad_sequences
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, concatenate, Reshape
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import time
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split



In [0]:
path_to_file = "/content/Chatbot_data.txt"

In [0]:
data = io.open(path_to_file,errors = 'ignore').read().strip().split('- - ')

In [0]:
raw_question = []
raw_ans = []
for i in range(len(data)):
    pair = data[i].split('  - ')
    for j in range(len(pair)-1):
        raw_question.append(pair[0])
        raw_ans.append(pair[j+1])


In [0]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = " ".join(text.split())
    return text

In [0]:
clean_questions = []
for question in raw_question:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in raw_ans:
    clean_answers.append(clean_text(answer))

In [0]:
question = []
ans = []
for i in range(len(clean_questions)):
  question.append('<start> '+ clean_questions[i] +' <end>')
for i in range(len(clean_answers)):
  ans.append('<start> '+ clean_answers[i] +' <end>')



In [0]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='',num_words=1000,oov_token='<oov>')

tokenizer.fit_on_texts(question)
question_seq = tokenizer.texts_to_sequences(question)
question_seq = tf.keras.preprocessing.sequence.pad_sequences(question_seq,padding='post')
tokenizer.fit_on_texts(ans)
ans_seq = tokenizer.texts_to_sequences(ans)
ans_seq = tf.keras.preprocessing.sequence.pad_sequences(ans_seq, padding = 'post')

In [0]:
len_input_seq = len(question_seq[0])
len_output_seq = len(ans_seq[0])
print(len(ans))

BUFFER_SIZE = len(question_seq)
batch_size = 64
BATCH_SIZE = 64
epochs = 10
steps_per_epoch = len(question_seq)//BATCH_SIZE
embedding_dim = 256
units = 256
vocab_inp_size = len(tokenizer.word_index)+1
vocab_tar_size = len(tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((question_seq, ans_seq)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)


9204


In [0]:
embedding_question = Embedding(vocab_inp_size,embedding_dim)
embedding_answer = Embedding(vocab_tar_size,embedding_dim)

In [0]:
enc_inputs = Input(shape = (len_input_seq,))
enc_inputs1 = embedding_question(enc_inputs)
encoder = LSTM(units, return_state=True,return_sequences=True)
enc_outputs, state_h, state_c = encoder(enc_inputs1)
enc_states = [state_h,state_c]


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)

    self.V1 = tf.keras.layers.Dense(1)

  def call(self, query1, values):
    query1_with_time_axis = tf.expand_dims(query1, 1)

    score1 = self.V1(tf.nn.tanh(self.W1(query1_with_time_axis) + self.W2(values)))

    attention_weights1 = tf.nn.softmax(score1, axis=1)

    context_vector1 = attention_weights1 * values

    context_vector1 = tf.reduce_sum(context_vector1, axis=1)

    return context_vector1


In [0]:
dec_inputs = Input(shape = (len_output_seq,))
dec_inputs1 = embedding_answer(dec_inputs)
#attention = BahdanauAttention(10)

#context_vec_1= BahdanauAttention(state_h,enc_outputs)
#dec_inputs = tf.concat([tf.expand_dim(context_vec_1,1),dec_inputs],axis = -1)

decode_lstm = LSTM(units,return_sequences=True,return_state=True)

dec_outputs1,_,_ = decode_lstm(dec_inputs1,enc_states)
#dec_outputs = tf.reshape(output, (-1, dec_outputs.shape[2]))
dec_reshape = Reshape((len_output_seq,))
dec_outputs1 = dec_reshape(dec_outputs1)
dec_dense = Dense(len_output_seq,activation = 'softmax')
dec_outputs = dec_dense(dec_outputs1)

In [0]:
model = Model([enc_inputs, dec_inputs], dec_outputs)


In [0]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    567808      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    567808      input_9[0][0]                    
____________________________________________________________________________________________

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
decoder_input_data = ans_seq

decoder_target_data = []
n = len(ans_seq)
for i in range(n):
  decoder_target_data.append(ans_seq[i][1:])

In [0]:
ans_seq[0]

In [0]:
decoder_target_data[0]

In [0]:

model.fit(input = [question_seq, decoder_input_data],output = decoder_target_data, epochs=epochs,batch_size=batch_size)